In [1]:
import time
import numpy as np   #计算库
import pandas as pd  #
# import pymysql #连接数据库
from sklearn.ensemble import RandomForestClassifier


from pyecharts.charts import Bar3D


# 导入文件
names = ['2015','2016','2017','2018','会员等级']
dates = [pd.read_excel('sales.xlsx',name = i) for i in names]

# 数据审查
for eachname,eachdata in zip(names,dates):
    print('----------------------'.format(eachname))
    print('overview','\n',eachdata.head(4))#展示数据的前四条  head 默认是五个  不写数字的话
    print('desc', '\n', eachdata.describe())#数据的描述信息
    print('NA records', eachdata.isnull().any(axis=1).sum())#缺失值记录
    print('Dtypes',eachdata.dtypes)#数据类型






----------------------
overview 
           会员ID         订单号       提交日期    订单金额
0  15278002468  3000304681 2015-01-01   499.0
1  39236378972  3000305791 2015-01-01  2588.0
2  38722039578  3000641787 2015-01-01   498.0
3  11049640063  3000798913 2015-01-01  1572.0
desc 
                会员ID           订单号           订单金额
count  3.077400e+04  3.077400e+04   30774.000000
mean   2.918779e+10  4.020414e+09     960.991161
std    1.385333e+10  2.630510e+08    2068.107231
min    2.670000e+02  3.000305e+09       0.500000
25%    1.944122e+10  3.885510e+09      59.000000
50%    3.746545e+10  4.117491e+09     139.000000
75%    3.923593e+10  4.234882e+09     899.000000
max    3.954613e+10  4.282025e+09  111750.000000
NA records 0
Dtypes 会员ID             int64
订单号              int64
提交日期    datetime64[ns]
订单金额           float64
dtype: object
----------------------
overview 
           会员ID         订单号       提交日期    订单金额
0  15278002468  3000304681 2015-01-01   499.0
1  39236378972  3000305791 2015-01-0

In [6]:
# 去除缺失值还有异常值
for ind,eachdata in enumerate(dates[:-1]):   #同时获得数据索引和值
    dates[ind] = eachdata.dropna()
    dates[ind] = eachdata[eachdata['订单金额']>1] #丢弃订单金额小于一的记录
    dates[ind]['max_year_date'] = eachdata['提交日期'].max() #获取一年中日期最大的值 方便后续RFM计算
# # 提交日期就是为了四年里面最大的时间 因为四年吗   增加一列

dates[1].head()

,会员ID,订单号,提交日期,订单金额,max_year_date
0,15278002468,3000304681,2015-01-01,499.0,2015-12-31
1,39236378972,3000305791,2015-01-01,2588.0,2015-12-31
2,38722039578,3000641787,2015-01-01,498.0,2015-12-31
3,11049640063,3000798913,2015-01-01,1572.0,2015-12-31
4,35038752292,3000821546,2015-01-01,10.1,2015-12-31


In [8]:
# 汇总所有的数据
data_sum = pd.concat(dates[:-1], axis=0)#处理数据时候少一张表  哪个是会员登记表 【：-1】这是切片
# 获取各自年份数据
# 接着增加一列   算出时间差值来
data_sum['date_interval'] = data_sum['max_year_date']-data_sum['提交日期']
data_sum['year'] = data_sum['提交日期'].dt.year
# dt就是时间类型的一个东西   有了这个你可以查询什么星期啊  几月啊  哪一年啊  后面的year就是一个属性值  跟字段没有关系
# 时间相减后都是datetime类型
# 时间间隔转化成数字
data_sum['date_interval'] = data_sum['date_interval'].apply(lambda x: x.days)

data_sum.head()

,会员ID,订单号,提交日期,订单金额,max_year_date,date_interval,year
0,15278002468,3000304681,2015-01-01,499.0,2015-12-31,364,2015
1,39236378972,3000305791,2015-01-01,2588.0,2015-12-31,364,2015
2,38722039578,3000641787,2015-01-01,498.0,2015-12-31,364,2015
3,11049640063,3000798913,2015-01-01,1572.0,2015-12-31,364,2015
4,35038752292,3000821546,2015-01-01,10.1,2015-12-31,364,2015


In [23]:

# 按照会员的id进行汇总
# 使用group分组  用year和会员id来做关联 as_index=FLASE是因为year和会员id不会作为index的列

rfm_gb = data_sum.groupby(['year', '会员ID'],as_index=False).agg(
    {
        'date_interval': 'min',
        '提交日期': 'count',
        '订单金额': 'sum',
    }
)
# rfm_gb.columns = ['year', '会员ID', 'r', 'f', 'm']

rfm_gb.head()


,year,会员ID,date_interval,提交日期,订单金额
0,2015,267,197,8,420.0
1,2015,282,251,4,118.8
2,2015,283,340,4,21592.0
3,2015,343,300,4,472.0
4,2015,525,37,12,852.0


In [13]:
# 查看数据的分布

desc_pd= rfm_gb.iloc[:,2:].describe().T
print(desc_pd)

     count         mean           std  min    25%    50%     75%       max
r  27611.0   177.761110    104.878641  0.0   84.0  178.0   267.0     364.0
f  27611.0     4.429249      5.269198  4.0    4.0    4.0     4.0     248.0
m  27611.0  4284.284161  10550.050645  6.0  264.0  672.0  3996.0  544560.0


In [18]:
r_bins = [0,84,178,267,364]
f_bins = [0,2,5,248]
m_bins = [0,264, 672, 3996,544560.0]
# 计算RFM因子权重   注意  因为时间的原因越小越好 所以应该在123三个维度里里面是倒序 321来计算
# 业务方面  就是算出权重之后他们自己做  一般都是越好的给一些优惠券免运费

# 匹配会员的等级还有RFM的得分
rfm_merge = pd.merge(rfm_gb,dates[-1],on='会员ID',how='inner')

TypeError: merge() got multiple values for argument 'how'

In [12]:

clf = RandomForestClassifier()#随机森里
clf = clf.fit(rfm_merge[['r','f','m']],rfm_merge['会员等级'])
weights = clf.feature_importances_

print('feature importance:',weights)


TypeError: merge() got multiple values for argument 'how'

AttributeError: 'DataFrame' object has no attribute 'read_excel'